In [46]:
# Embassaments catalans

import pandas as pd
import numpy as np

data = pd.read_csv("Quantitat_d_aigua_als_embassaments_de_les_Conques_Internes_de_Catalunya_20240607.csv")

In [47]:
# EXPLORACIÓ INICIAL DADES


# Informació general atributs
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80316 entries, 0 to 80315
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Dia                             80316 non-null  object 
 1   Estació                         80316 non-null  object 
 2   Nivell absolut (msnm)           80284 non-null  float64
 3   Percentatge volum embassat (%)  80316 non-null  float64
 4   Volum embassat (hm3)            80316 non-null  float64
dtypes: float64(3), object(2)
memory usage: 3.1+ MB


In [48]:
# Resum descriptiu de dades
data.describe()

,Nivell absolut (msnm),Percentatge volum embassat (%),Volum embassat (hm3)
count,80284.000000,80316.000000,80316.000000
mean,401.417327,66.184408,50.987741
std,215.870703,27.475762,56.814232
min,96.790000,0.900000,0.110000
25%,205.540000,43.800000,4.820000
50%,413.205000,70.700000,25.360000
75%,528.550000,90.500000,79.010000
max,806.660000,151.700000,247.440000


In [49]:
# Quants valors únics "d'Estació" disposem (embassaments)

data["Estació"].nunique()

9

In [50]:
# De quants dies tenim dades

data["Dia"].nunique()

8924

In [51]:
# Mostrem les primeres entrades de les dades
data.head(10)

,Dia,Estació,Nivell absolut (msnm),Percentatge volum embassat (%),Volum embassat (hm3)
0,06/06/2024,Embassament de Sau (Vilanova de Sau),398.16,30.6,50.59
1,06/06/2024,Embassament de la Baells (Cercs),609.96,47.0,51.48
2,06/06/2024,Embassament de Foix (Castellet i la Gornal),98.47,68.8,2.57
3,06/06/2024,Embassament de Sant Ponç (Clariana de Cardener),515.01,34.3,8.36
4,06/06/2024,Embassament de Susqueda (Osor),311.96,31.7,73.78
5,06/06/2024,Embassament de la Llosa del Cavall (Navès),774.96,26.8,21.47
6,06/06/2024,Embassament de Riudecanyes,194.37,2.5,0.13
7,06/06/2024,Embassament de Siurana (Cornudella de Montsant),452.44,2.3,0.28
8,06/06/2024,Embassament de Darnius Boadella (Darnius),138.85,22.6,13.80
9,05/06/2024,Embassament de Riudecanyes,194.38,2.5,0.13


**Modificació dades per al primer gràfic**

Per al primer gràfic, necesitem modificar una mica les dades per ajustar la seva estructura. El que fem es separar el nom de l'embassament i la població, així com li afegim la seva geolocalització a les variables lat i lon que utilitzarem per situar-los geogràficament en el mapa. Per últim estipulem a la conca on pertanyen els embassaments.

La idea és generar un nou dataframe amb les dades justes per al gràfic que volem, el nom de l'embassament en curt (sense embassament al davant) la població, la latitud, longitud i la conca. En aquí he afegit els embassaments també de la conca de l'Ebre per a reconeixer millor la distribució general i el context dels embassaments, tot i que més endevant tractarem només de les Conques Internes que són de les que disposem dades.

In [52]:
embassament = ["Foix", "La Baells", "Sau", "Darnius Boadella", "Susqueda", "Sant Ponç", "Llosa del Cavall", "Siurana", "Riudecanyes","Baserca","Camarasa","Canelles","Cavallers","Guiamets","Oliana","Rialb","Riba-roja d'Ebre","Talarn", "Terradets"]
poblacio = ["Castellet i la Gornal", "Cercs", "Vilanova de Sau", "Darnius", "Osor", "Clariana de Cardener", "Navès", "Cornudella de Montsant", "Riudecanyes"]
lat = [41.256284, 42.121933,41.974197,42.340833,41.9792,41.962919,42.0998,41.2497,41.133521,42.588039, 41.90806,41.978611,42.589937,41.0988,42.098969,41.940372,41.2436,42.179719,42.055]
lon = [1.64972, 1.87835,2.384292,2.833333,2.52722,1.605189,1.58285,0.913803,0.958907,0.760375, 0.886862,0.612222,0.856305,0.751617,1.296422,1.193775,0.4328,0.913061,0.888]
conca = ["Interna","Interna","Interna","Interna","Interna","Interna","Interna","Interna","Interna", "Ebre","Ebre","Ebre","Ebre","Ebre","Ebre","Ebre","Ebre","Ebre", "Ebre"]


**Càlcul del volum total de cada embassament:**

Com que necesitem calcular el volum total en hm3 de cada embassament, en comptes de buscar manualment per internet els valors de cadascún, el que fem és aprofitar les dades de que disposem, percentatge de volum i volum actual, per generar unes dades agregades i aconseguir el volum total. D'aquesta manera, a part d'aconseguir fer-ho automàticament també aconseguim que el valor de volum total sigui més coherent amb les dades suministrades (ja que és calcula a elles mateixes).

Com que sabem que un valor porcentual es calcula com: (valor_total * percentatge) / 100 = valor_percentual l'únic que hem de fer es aïllar aquesta i tenim que l'equació ens queda: valor_total = (valor_percentual / porcentatge) * 100 que és el que apliquem a continuació.

In [53]:
# Llistat dels noms de tots els 9 embassaments continguts a les nostres dades
noms_embassaments = ["Embassament de Foix (Castellet i la Gornal)","Embassament de la Baells (Cercs)","Embassament de Sau (Vilanova de Sau)","Embassament de Darnius Boadella (Darnius)","Embassament de Susqueda (Osor)","Embassament de Sant Ponç (Clariana de Cardener)","Embassament de la Llosa del Cavall (Navès)","Embassament de Siurana (Cornudella de Montsant)","Embassament de Riudecanyes"]
volum_total = []
volum_actual = []
volum_actual_percentatge = []

# Agregació de dades internes (volum total de cada embassament)
# iteració per a cada embassament per recalcular el volum total de cada un en base als valors de volum actual i porcentatge actual
for nom in noms_embassaments:

  volum_embassat = data[data.Estació == nom].iloc[0]["Volum embassat (hm3)"]
  volum_percentatge = data[data.Estació == nom].iloc[0]["Percentatge volum embassat (%)"]
  volum_total_embassament = round((volum_embassat / volum_percentatge) * 100, 2)

  volum_total.append(volum_total_embassament)
  volum_actual.append(volum_embassat)
  volum_actual_percentatge.append(volum_percentatge)


In [54]:
# insereixo manualment el volum total dels embassaments de la conca de l'Ebre (no es disposen dades obertes, només de les internes)
volum_total_ebre = [21.9,163.4,679.3,16.1,9.7,101.1,403.6,209.6,226.7,33.2]
volum_total = volum_total + volum_total_ebre

# he d'acabar d'omplir amb valors nuls (que no tenim de les conques de l'Ebre) per que s'executi correctament la funció zip
for i in range(9,19):
  poblacio.append(None)
  volum_actual.append(None)
  volum_actual_percentatge.append(None)

In [55]:
# Amb la funció zip juntem les 8 llistes amb les dades previament establertes
# Amb el zip de dades fet, generem un dataframe amb la informació bàsica sobre cada embassament

list_data = zip(embassament, poblacio, volum_actual_percentatge, volum_actual, volum_total, conca, lat, lon)
df_embassaments = pd.DataFrame(list_data, columns=["embassament", "poblacio", "volum actual percentatge %", "volum actual hm3", "volum total hm3", "conca", "lat", "lon"])

In [56]:
# Comprovació correcte establiment del dataframe d'informació d'embassaments
df_embassaments.head(20)

,embassament,poblacio,volum actual percentatge %,volum actual hm3,volum total hm3,conca,lat,lon
0,Foix,Castellet i la Gornal,68.8,2.57,3.74,Interna,41.256284,1.649720
1,La Baells,Cercs,47.0,51.48,109.53,Interna,42.121933,1.878350
2,Sau,Vilanova de Sau,30.6,50.59,165.33,Interna,41.974197,2.384292
3,Darnius Boadella,Darnius,22.6,13.80,61.06,Interna,42.340833,2.833333
4,Susqueda,Osor,31.7,73.78,232.74,Interna,41.979200,2.527220
5,Sant Ponç,Clariana de Cardener,34.3,8.36,24.37,Interna,41.962919,1.605189
6,Llosa del Cavall,Navès,26.8,21.47,80.11,Interna,42.099800,1.582850
7,Siurana,Cornudella de Montsant,2.3,0.28,12.17,Interna,41.249700,0.913803
8,Riudecanyes,Riudecanyes,2.5,0.13,5.20,Interna,41.133521,0.958907
9,Baserca,None,NaN,NaN,21.90,Ebre,42.588039,0.760375


In [57]:
# Generació de la primera visualització
# Mostrar el Mapa interactiu de Ctalunya amb la geolocalització dels principals 9 embassaments corresponents a les conques internes, així com el seu volum total i la localitat

import pandas as pd
import plotly.express as px

# aquest és el meu token de la llibreria mapbox per poder generar el mapa amb ells
mapbox_acces_token = "pk.eyJ1IjoiZ2dpcm9uZXNyb3Z1b2MiLCJhIjoiY2x4MzlldjhhMDJoazJrc2JqcTQ5dm13dSJ9.eOUM1nIGb4BOq7hqU6yaxg"

# els principals paràmetres de cofniguració del mapa
fig = px.scatter_mapbox(df_embassaments,
                        lat="lat",
                        lon="lon",
                        hover_name="embassament",
                        # paràmetre per posar el nom de l'embassament sobre el mapa
                        text = "embassament",
                        # mostrem algunes dades quan es posi el cursor sobre cada embassament
                        hover_data=["poblacio", "volum actual percentatge %", "volum actual hm3", "volum total hm3"],
                        # marquem en color blau i ocre si es de la Conca Interna o de l'Ebre
                        color=df_embassaments["conca"],
                        color_discrete_sequence=["royalblue", "peru"],
                        # he decidit una mida de 1000x1000 del gràfic, prou gran
                        width=1000,
                        height=1000)

# paràmetres pel tamany, estil i color dels marcadors (cercles d'embassaments)
fig.update_traces(marker={"size": 18, "symbol": "circle" },
                  textposition = "bottom right",
                  mode="markers+text")

fig.update_layout(title="Mapa d'embassaments a Catalunya (capacitats superiors a 2 hm3)",
    mapbox=dict(
        accesstoken=mapbox_acces_token,
        bearing=0,
        center=dict(
            lat=41.72,
            lon=1.8),
        style="outdoors",
        zoom=7.55
    ),
)

fig.show()

In [60]:
import plotly.graph_objects as go

emb=df_embassaments["embassament"]

# preparació de la variable amb els colors
colors_fill = ['royalblue',] * 9
colors_empty = ['lightslategray',] * 9

# càlcul del volum restant entre el total i l'actual per pintar la zona "buida"
volum_faltant = []
for i in range(0,9):
  volum_faltant.append(round(df_embassaments["volum total hm3"][i] - df_embassaments["volum actual hm3"][i], 2))

# generació principal del gràfic de barres amb dues barres sobreposades
fig = go.Figure(
    data=[
      go.Bar(name='Volum actual hm3', x=emb, y=df_embassaments["volum actual hm3"][0:9], marker_color=colors_fill, text=df_embassaments["volum actual hm3"][0:9]),
      go.Bar(name='Volum total hm3', x=emb, y=volum_faltant, marker_color=colors_empty, text=df_embassaments["volum total hm3"][0:9]),
    ],

)

# Configurar la barra com a "stacked" o agregades
fig.update_layout(barmode='stack',
                  title="Estat actual dels embassaments de les Conques Internes de Catalunya",
                  xaxis_title="Embassaments",
                  yaxis_title="Volum d'aigua en Hm3",
                  width=1000,
                  height=900)

# Ordenar embassaments de més petit a més gran
fig.update_xaxes(categoryorder='total ascending')


fig.show()

In [63]:
data.head(7)

,Dia,Estació,Nivell absolut (msnm),Percentatge volum embassat (%),Volum embassat (hm3)
0,06/06/2024,Embassament de Sau (Vilanova de Sau),398.16,30.6,50.59
1,06/06/2024,Embassament de la Baells (Cercs),609.96,47.0,51.48
2,06/06/2024,Embassament de Foix (Castellet i la Gornal),98.47,68.8,2.57
3,06/06/2024,Embassament de Sant Ponç (Clariana de Cardener),515.01,34.3,8.36
4,06/06/2024,Embassament de Susqueda (Osor),311.96,31.7,73.78
5,06/06/2024,Embassament de la Llosa del Cavall (Navès),774.96,26.8,21.47
6,06/06/2024,Embassament de Riudecanyes,194.37,2.5,0.13


In [64]:
# asegurem el format datetime de la columna dia
data['Dia'] = pd.to_datetime(data['Dia'],format='%d/%m/%Y')

# agrupem els els valors dels volums d'aigua per mesos "M"
data_monthly = data[["Dia","Volum embassat (hm3)"]].groupby(pd.Grouper(key="Dia", axis=0, freq="D")).sum()
#df_monthly_percent = data[["Dia","Volum embassat (hm3)"]].groupby(pd.Grouper(key="Dia", axis=0, freq="M")).mean()

data_monthly = data_monthly.reset_index()


In [65]:
# seleccionem l'ultim dia de cada mes
data_monthly = data_monthly.groupby([data_monthly['Dia'].dt.year, data_monthly['Dia'].dt.month], as_index=False).last()

# formatejo de dies a mes perque en el gràfic de barres és veurà millor (barres més amples i clares per mesos)
data_monthly["Dia"] = pd.to_datetime(data_monthly["Dia"], format='%Y00%m').apply(lambda x: x.strftime('%Y-%m'))

# Com tenim el volum actual en hm3, i volem visualitzar en % del volum total (ja que queda més clar per l'usuari)
# apliquem la fòrmula (volum_actual / volum_total) * 100 a les dades i ja tenim el % del total d'embassaments
data_monthly["Volum embassat (hm3)"] = data_monthly["Volum embassat (hm3)"].apply(lambda x: (x / 700)*100 )

data_monthly

,Dia,Volum embassat (hm3)
0,2000-01,50.624286
1,2000-02,47.782857
2,2000-03,43.605714
3,2000-04,46.655714
4,2000-05,49.885714
...,...,...
289,2024-02,14.390000
290,2024-03,16.140000
291,2024-04,19.450000
292,2024-05,29.841429


In [69]:
# Renomrem la columna per representar corectament l'escala (no esta an hm3, sino en porcentatge)
data_monthly = data_monthly.rename(columns={"Volum embassat (hm3)": "Volum embassat %"})

In [70]:
#agafem només els últims 125 mesos, per mostrar dades dels últims 10 anys (a partir del 2014)
data_monthly = data_monthly.tail(126)

In [71]:
import plotly.express as px

fig = px.bar(data_monthly,
             x="Dia",
             y="Volum embassat %",
             )

fig.update_layout(title="Evolució del nivell (% percentatge) dels embassaments de les Conques Internes de Catalunya (2014-2024) a dia 7/6/2024",
                  xaxis_title="Evolució mensual (2014-2024)",
                  yaxis_title="Percentatge % del nivell dels embassaments",
                  )

fig.show()